In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

import torchvision.datasets as datasets
import torchvision
import torchvision.transforms as transforms

# general
import pandas as pd 
import numpy as np 
import itertools
from scipy.special import comb
from cvxopt import matrix, solvers, spdiag
import copy
import pickle
import sys
import time
import os

# scikit-learn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Keras / Tensorflow Libraries
import tensorflow.compat.v1 as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Input, Flatten, Dense, LeakyReLU, Dropout, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
import tensorflow.keras.utils as np_utils
import tensorflow

from utility_functions import *
from pubfig83_utility import *
from deepset import *

import argparse


def getSelectedAcc(rank, target_size, x_train, y_train, x_val, y_val, utilityFunc, interval=50):
  if rank is None or utilityFunc is None:
    return None
  ret = np.zeros(int(target_size/interval))
  for i in range(1, int(target_size/interval)+1):
    ret[i-1] = utilityFunc(x_train[rank[::-1][interval*(i):]], y_train[rank[::-1][interval*(i):]], x_val, y_val)
  return ret

print(tf.config.list_physical_devices('GPU'))

parser = argparse.ArgumentParser('')

parser.add_argument('--attack_type', type=str)

parser.add_argument('--lc', action='store_true')
parser.add_argument('--sv', action='store_true')
parser.add_argument('--loo', action='store_true')
parser.add_argument('--tmc', action='store_true')
parser.add_argument('--gshap', action='store_true')
parser.add_argument('--inf', action='store_true')
parser.add_argument('--tracin_self', action='store_true')
parser.add_argument('--tracin_clean', action='store_true')
parser.add_argument('--knn', action='store_true')
parser.add_argument('--deepset', action='store_true')
parser.add_argument('--random', action='store_true')

args = parser.parse_args()

rank_coll_dir = 'rank-collections/'


post_fix = ''
if args.lc:
  post_fix += 'LC_'
if args.sv:
  post_fix += 'SV_'
if args.loo:
  post_fix += 'LOO_'
if args.tmc:
  post_fix += 'TMC_'
if args.gshap:
  post_fix += 'GSHAP_'
if args.inf:
  post_fix += 'INF_'
if args.tracin_self:
  post_fix += 'TRACINSELF_'
if args.tracin_clean:
  post_fix += 'TRACINCLEAN_'
if args.knn:
  post_fix += 'KNN_'
if args.random:
  post_fix += 'RANDOM_'
if args.deepset:
  post_fix += 'DEEPSET'


print('POST FIX:', post_fix)

attack_type = args.attack_type



if attack_type == 'CIFAR_Backdoor':

  data_file = 'backdoor_cifar_N50000_poison2500_val1000.data'
  x_train_few, y_train_few, x_val_poi, y_val_poi, x_val, y_val = pickle.load( 
    open('low-quality-data/'+data_file, 'rb') )

  y_train_few = y_train_few.reshape(-1)
  y_val_poi = y_val_poi.reshape(-1)
  y_val = y_val.reshape(-1)

  n_data_deepset = 2000
  n_epoch = 16
  n_set = 64
  n_hext = 64
  n_hreg = 64
  LR = 1e-4

  deepset_dir = 'saved-deepset/Rebuttal_Backdoor_CIFAR_N{}_DataSeed100_Nepoch{}_Nset{}_Next{}_Nreg{}_LR{}.state_dict'.format(
    n_data_deepset, n_epoch, n_set, n_hext, n_hreg, LR)

  func_data_to_acc = None
  # func_data_to_atkacc = torch_cifar_smallCNN_data_to_acc_multiple
  func_data_to_atkacc = torch_cifar_vit_data_to_acc


elif attack_type == '...'



acc_coll_dir = rank_coll_dir+'ICMLRebuttal2_{}_{}.acccoll'.format(attack_type, post_fix)

#x_train_few, y_train_few = x_train_few[:n_data_deepset], y_train_few[:n_data_deepset]

n_data = x_train_few.shape[0]
target_size = x_train_few.shape[0]


if args.inf:

  [score] = pickle.load( open("saved-samples/ICML_{}_INF.sample".format(attack_type), 'rb') )
  inf_rank = np.argsort(score)

if args.tracin_self:

  [score] = pickle.load( open("saved-samples/ICML_{}_TRACINSELF.sample".format(attack_type), 'rb') )
  tracinself_rank = np.argsort(score)[::-1]

if args.tracin_clean:

  [score] = pickle.load( open("saved-samples/ICML_{}_TRACINCLEAN.sample".format(attack_type), 'rb') )
  tracinclean_rank = np.argsort(score)

if args.knn:

  [score] = pickle.load( open("saved-samples/ICML_{}_KNN.sample".format(attack_type), 'rb') )
  knn_rank = np.argsort(score)




if not args.lc:
  lc_rank = None
if not args.sv:
  sv_rank = None
if not args.loo:
  loo_rank = None
if not args.tmc:
  tmc_rank = None
if not args.gshap:
  gshap_rank = None
if not args.inf:
  inf_rank = None
if not args.tracin_self:
  tracinself_rank = None
if not args.tracin_clean:
  tracinclean_rank = None
if not args.knn:
  knn_rank = None
if not args.deepset:
  deepset_rank = None
if not args.random:
  random_rank = None


if args.deepset:

  if attack_type=='CIFAR_Mislabel' or attack_type=='CIFAR_Poison' or attack_type=='CIFAR_Noisy':
    opt = tf.train.AdamOptimizer()
    extractor = CifarExtractor.build(numChannels=3, imgRows=32, imgCols=32, numClasses=10)
    extractor.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    load_status = extractor.load_weights('cifar_featureExtractor.ckpt')
    lastLayerOp = K.function([extractor.layers[0].input], [extractor.layers[-5].output])
    x_train_few_cnnFeature = extractFeatures(lastLayerOp, x_train_few)

    print('FeatureExtractor', x_train_few_cnnFeature.shape)

    n_cls = 10
    eps = 1e-3

    if len(y_train_few.shape) < 2 or y_train_few.shape[1]!=10:
      y_train_few_hot = np_utils.to_categorical(y_train_few, 10)
    else:
      y_train_few_hot = y_train_few

  elif attack_type[:5]=='CIFAR' or attack_type[:6]=='DOGCAT':

    extractor_savename = 'saved-deepset/Rebuttal_Backdoor_CIFAR_Extractor.state_dict'

    extractor = SmallCNN_CIFAR().cuda()
    extractor.load_state_dict( torch.load( extractor_savename ) )

    if x_train_few.shape[1]==32:
        x_train_few_cf = np.moveaxis(x_train_few, 3, 1)
    tensor_x = torch.Tensor(x_train_few_cf).cuda()
    x_train_few_cnnFeature = extractor.getFeature(tensor_x)
    x_train_few_cnnFeature = x_train_few_cnnFeature.cpu().detach().numpy()

    print('FeatureExtractor', x_train_few_cnnFeature.shape)

    n_cls = 10
    eps = 1e-3

    if len(y_train_few.shape) < 2 or y_train_few.shape[1]!=10:
      y_train_few_hot = np_utils.to_categorical(y_train_few, 10)
    else:
      y_train_few_hot = y_train_few


  elif attack_type[:4]=='SPAM':

    from sklearn.feature_selection import SelectPercentile
    from sklearn.feature_selection import chi2
    selector = SelectPercentile(score_func=chi2, percentile=10)
    x_train_few_cnnFeature = selector.fit_transform(x_train_clean, y_train_clean)
    x_train_few_cnnFeature = x_train_few_cnnFeature[train_ind]
    n_cls = 2
    y_train_few_hot = y_train_few.reshape((len(y_train_few), 1))

    eps = 1e-3


  elif attack_type[:5]=='MNIST':
    opt = tf.train.AdamOptimizer()
    extractor = KerasLeNet.build(numChannels=1, imgRows=28, imgCols=28, numClasses=10)
    extractor.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    load_status = extractor.load_weights('mnist_featureExtractor.h5')
    lastLayerOp = K.function([extractor.layers[0].input], [extractor.layers[6].output])
    x_train_few_cnnFeature = extractFeatures(lastLayerOp, x_train_few)
    n_cls = 10
    eps = 1e-3

    if len(y_train_few.shape) < 2 or y_train_few.shape[1]!=10:
      y_train_few_hot = np_utils.to_categorical(y_train_few, 10)
    else:
      y_train_few_hot = y_train_few


  elif attack_type[:6]=='PUBFIG':
    opt = tf.train.AdamOptimizer()
    extractor = CifarExtractor.build(numChannels=3, imgRows=32, imgCols=32, numClasses=83)
    extractor.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    load_status = extractor.load_weights('pubfig_featureExtractor.h5')
    lastLayerOp = K.function([extractor.layers[0].input], [extractor.layers[-5].output])
    x_train_few_cnnFeature = extractFeatures(lastLayerOp, x_train_few)

    n_cls = 83
    y_train_few_hot = np_utils.to_categorical(y_train_few)
    eps = 1e-3
    




deepset_rank_coll = np.zeros((10, n_data))
random_rank_coll = np.zeros((10, n_data))
acc_coll = {}
atkacc_coll = {}

acc_coll['deepset'] = []
acc_coll['lc'] = []
acc_coll['sv'] = []
acc_coll['loo'] = []
acc_coll['tmc'] = []
acc_coll['gshap'] = []
acc_coll['inf'] = []
acc_coll['tracinself'] = []
acc_coll['tracinclean'] = []
acc_coll['knn'] = []
acc_coll['random'] = []

atkacc_coll['deepset'] = []
atkacc_coll['lc'] = []
atkacc_coll['sv'] = []
atkacc_coll['loo'] = []
atkacc_coll['tmc'] = []
atkacc_coll['gshap'] = []
atkacc_coll['inf'] = []
atkacc_coll['tracinself'] = []
atkacc_coll['tracinclean'] = []
atkacc_coll['knn'] = []
atkacc_coll['random'] = []




# Load (x_bad, y_bad)
# Load Data Utility Model





for select_seed in range(10):
#for select_seed in range(1):

  if args.deepset and attack_type[:6]!='PUBFIG':

    n_block = int(n_data/n_data_deepset)
    deepset_rank_matrix = np.zeros((n_block, n_data_deepset))
    random_perm = np.random.permutation(range(n_data))

    for i in range(n_block):
      
      random_ind = random_perm[n_data_deepset*(i):n_data_deepset*(i+1)]

      if attack_type == 'MNIST_Noisy':

        _, deepset_rank_small, _ = findMostValuableSample_deepset_stochasticgreedy_OLD(deepset_model.model, 
                                                                                       x_train_few_cnnFeature[random_ind], 
                                                                                       n_data_deepset, epsilon=eps, seed=select_seed)
      else:
        _, deepset_rank_small, _ , _ = findMostValuableSample_deepset_stochasticgreedy(deepset_model.model, 
                                                                                       x_train_few_cnnFeature[random_ind], 
                                                                                       y_train_few_hot[random_ind], 
                                                                                       n_data_deepset, epsilon=eps, seed=select_seed)

      deepset_rank_matrix[i] = random_ind[deepset_rank_small]

    deepset_rank = ((deepset_rank_matrix.T).reshape(-1)).astype(int)
    deepset_rank_coll[select_seed] = deepset_rank

  if args.random:
    random_rank = np.random.permutation(range(n_data))
    random_rank_coll[select_seed] = random_rank

  if args.deepset and attack_type[:6]=='PUBFIG':
    deepset_rank = np.random.permutation(range(n_data))
    deepset_rank_coll[select_seed] = deepset_rank


  half_size = int(target_size / 2)
  interval = int(half_size / 10)
  # interval = int(half_size / 5)

  print('Select Seed', select_seed)

  deepset_acc = getSelectedAcc(deepset_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)  
  
  lc_acc = getSelectedAcc(lc_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  sv_acc = getSelectedAcc(sv_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  loo_acc = getSelectedAcc(loo_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  tmc_acc = getSelectedAcc(tmc_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  gshap_acc = getSelectedAcc(gshap_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  inf_acc = getSelectedAcc(inf_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  tracinself_acc = getSelectedAcc(tracinself_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  tracinclean_acc = getSelectedAcc(tracinclean_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  
  knn_acc = getSelectedAcc(knn_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)
  
  # random_acc = getSelectedAcc(random_rank, half_size, x_train_few, y_train_few, x_val, y_val, func_data_to_acc, interval)

  acc_coll['deepset'].append(deepset_acc)
  acc_coll['lc'].append(lc_acc)
  acc_coll['sv'].append(sv_acc)
  acc_coll['loo'].append(loo_acc)
  acc_coll['tmc'].append(tmc_acc)
  acc_coll['gshap'].append(gshap_acc)
  acc_coll['inf'].append(inf_acc)
  acc_coll['tracinself'].append(tracinself_acc)
  acc_coll['tracinclean'].append(tracinclean_acc)
  acc_coll['knn'].append(knn_acc)
  acc_coll['random'].append(random_acc)

  deepset_acc = getSelectedAcc(deepset_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  lc_acc = getSelectedAcc(lc_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  sv_acc = getSelectedAcc(sv_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  loo_acc = getSelectedAcc(loo_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  tmc_acc = getSelectedAcc(tmc_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  gshap_acc = getSelectedAcc(gshap_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  inf_acc = getSelectedAcc(inf_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  tracinself_acc = getSelectedAcc(tracinself_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  tracinclean_acc = getSelectedAcc(tracinclean_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  knn_acc = getSelectedAcc(knn_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)
  random_acc = getSelectedAcc(random_rank, half_size, x_train_few, y_train_few, x_val_poi, y_val_poi, func_data_to_atkacc, interval)

  atkacc_coll['deepset'].append(deepset_acc)
  atkacc_coll['lc'].append(lc_acc)
  atkacc_coll['sv'].append(sv_acc)
  atkacc_coll['loo'].append(loo_acc)
  atkacc_coll['tmc'].append(tmc_acc)
  atkacc_coll['gshap'].append(gshap_acc)
  atkacc_coll['inf'].append(inf_acc)
  atkacc_coll['tracinself'].append(tracinself_acc)
  atkacc_coll['tracinclean'].append(tracinclean_acc)
  atkacc_coll['knn'].append(knn_acc)
  atkacc_coll['random'].append(random_acc)

  pickle.dump([deepset_rank_coll, random_rank_coll, acc_coll, atkacc_coll], open(acc_coll_dir, 'wb') )

  print('save!!!')








